# pandas (continued)

**If you are working on the cloud (for example Google Colab):** You will need to first run the line of code below. It will pull the files into your workspace. 

In [ ]:
!wget https://raw.githubusercontent.com/nuitrcs/python_workshops_datarepo/refs/heads/main/wnba-team-elo-ratings.csv
!wget https://raw.githubusercontent.com/nuitrcs/python_workshops_datarepo/refs/heads/main/pigeonRacing.txt

The files should now appear in your filetree in the same directory where this notebook is located. You should see them in your filetree. Look for `wnba-team-elo-ratings.csv` and `pigeonRacing.txt`.
<br><br>**If you are working locally on your own computer:** The files are here in the same repo where this notebook is located. You should see them in your filetree. Look for `wnba-team-elo-ratings.csv` and `pigeonRacing.txt`.

<br><br>First, we will import pandas into this notebook.

In [ ]:
import pandas as pd

# <br><br>6.0 Part Six: working with missing data
Let's load the "pigeonRacing.txt" file.

In [ ]:
df = pd.read_csv("pigeonRacing.txt", delimiter="\t")

<br>Let's take a look at our dataset:

In [ ]:
df.head()

<br>Let's look at a larger sample of our dataset:

In [ ]:
df.sample(10)

<br>Missing values are represented as a special pandas object: **`NaN`**. It stands for Not a Number. **The NaN value will let you complete numerical calculations on your data - it will ignore the value, so it will not include the value in calculations.**

In [ ]:
df["Average Birdage"].mean()

## <br><br>6.1 Finding missing data

We can look for missing data in our DataFrame using `isnull()`.

In [ ]:
df.isnull()

<br><br>We can do the opposite - look for all values that aren't missing - using `notnull()`.

In [ ]:
df.notnull()

<br><br>This would be hard to sort through in a big DataFrame. We can build upon this data frame to count the number of missing values in each column. We do this by using the aggregation method `.sum()`.

In [ ]:
df.isnull().sum()

This works because in Python, `True` evaluates to `1` and `False` evaluates to `0`.

Similarly, you can get number of missing values in a specific column by selecting the column first.

In [ ]:
df["Color"].isnull().sum()

We have two columns with missing data - one is numerical data (Average Birdage) and one is categorial (Color).

In [ ]:
df.dtypes

We're going to walk through a few common ways to deal with missing data. 
<br><br>First, let's check the number of rows and columns in our original DataFrame to compare later:

In [ ]:
df.shape

## <br><br>6.2 Drop all rows with NaN

Let's drop every row with any NaN value using the `dropna()` function.

In [ ]:
df_no_nan_rows = df.dropna()
df_no_nan_rows.shape

How many rows were removed?

In [ ]:
df.shape[0] - df_no_nan_rows.shape[0]

## <br><br>6.3 Drop all columns with NaN

Same function, but we add the keyword argument for columns. By default, `.dropna()` removed rows because `axis=0` was the default argument, which removes rows. To remove columns, we use `axis=1`.

In [ ]:
df_no_nan_columns = df.dropna(axis=1)
df_no_nan_columns.shape

How many columns were removed?

In [ ]:
df.shape[1] - df_no_nan_columns.shape[1]

## <br><br>6.4 Drop rows with NaN in a particular column

At first, you might try to use the regular `drop()` function with a boolean, but this won't work:

In [ ]:
df_drop_rows_nan_color = df.drop(df["Color"].isnull())
df_drop_rows_nan_color.shape

<br>Instead, we can create a subset of the DataFrame that includes only rows that are not null in the Color column. We are using the pandas function `notnull()` to return any non-NaN values. Instead of adding the function to the end of the DataFrame or the end of the column, we use the function at the end of the Series object we want to search through, and then use that as the index on the whole DataFrame. 

In [ ]:
df_drop_rows_nan_color = df[df["Color"].notnull()]
df_drop_rows_nan_color.shape

This is equivalent to creating a boolean series object that indicates which rows are missing for the `"Color"` column:

In [ ]:
missing_color_rows = df["Color"].notnull()
df_drop_rows_nan_color = df[missing_color_rows]
df_drop_rows_nan_color.shape

How many rows did we drop?

In [ ]:
df.shape[0] - df_drop_rows_nan_color.shape[0]

## <br><br>6.5 Replace all NaNs with zero

Pandas has a `fillna()` function that will work over the entire DataFrame:

In [ ]:
df_fill_zeros = df.fillna(0)
df_fill_zeros.shape

Let's look at all rows with missing `"Color"` or `"Average Birdage"` variable.

In [ ]:
row_select = df["Color"].isnull() | df["Average Birdage"].isnull() # rows with missing values in either column
df_fill_zeros[row_select] # display rows with filled values

<br>This replaced NaNs in both the Colors and Average Birdage columns with 0. That might not be what you want to do.

## <br><br>6.6 Replace NaNs in only one column with the mean value of that column.

This one is tricky. I don't expect you to remember how to do this. As with many things in Pandas, it isn't always intuitive how to solve every unique situation. Don't be afraid to search for solutions and use documentation that's available online.

<br>First, we need to make a copy of the original DataFrame because we are going to reference the new version on the left side of our assignment operator. If we don't do this, we would have to change the original df.
<br><br>Then I am going to reassign the Average Birdage column to the same column, but with the `fillna()` function applied. I am going to fill the NaNs with the mean value of that column in the original DataFrame, so that we're not self-refrencing the column as it's being changed.

In [ ]:
print("Average Birdage mean: " + str(df["Average Birdage"].mean()))

In [ ]:
df_mean_birdage = df.copy()
df_mean_birdage["Average Birdage"] = df_mean_birdage["Average Birdage"].fillna(df["Average Birdage"].mean())
df_mean_birdage[df["Average Birdage"].isnull()]

### <br><br>Exercise 6.6

Write code to replace any missing data in the `Color` column with the string "BB". I've started for you by making a copy of the DataFrame to play with.

In [ ]:
df_replace_colors = df.copy()

# <br><br>7.0 Part Seven: data types

Our sample dataset for this section was taken from FiveThirtyEight. It contains game data for WNBA games since 1997.

In [ ]:
df = pd.read_csv("wnba-team-elo-ratings.csv")

In [ ]:
df.head()

## <br><br>7.1 Checking and changing data types

<br><br>We can check the data type of all columns using the `dtypes` attribute.

In [ ]:
df.dtypes

<br>An `object` is a string or a column with mixed datatypes.

We can change the data type of a column using the method function `astype()`. Let's change the elo ratings from integers to floats. Since we only want the change applied to one column, we have to include that column on the left side of our assignment operator:

In [ ]:
df["elo1_pre"] = df["elo1_pre"].astype("float")
df.dtypes

<br>To change multiple columns, such as elo columns, we can pass a dictionary with column name as the key and the object type (in string) as the value.

In [ ]:
df = df.astype({"elo1_pre": "float", "elo2_pre": "float", "elo1_post": "float", "elo2_post": "float"})

### <br><br>Exercise 7.1

Write code to change the `season` column from an integer to an `object`:

In [ ]:
df.dtypes

Write code to change the playoff column from an integer to a boolean (try `bool`):

In [ ]:
df.sample(5)

## <br><br><br>7.2 Working with dates and times

A `datetime` is a special Python data type. To convert a column to a datetime object, you use a different function, `to_datetime`.

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.sample(5)

In [ ]:
df.dtypes

<br>The format shown is YYYY-MM-DD. If a time is provided, it will also be converted.
<br><br>Once a column is converted to a datetime object, you can sample that column based on several date formats:

In [ ]:
df.loc[df["date"] == "10-6-2019", :]

In [ ]:
df.loc[df["date"] == "10/6/19", :]

In [ ]:
df.loc[df["date"] == "Oct 6 2019", :]

<br><br>You can also use other operators with datetime objects:

In [ ]:
df.loc[df["date"] > "Oct 6 2019", :]

### <br><br>Exercise 7.2

Pick a random day between 2012 and 2019 and see if any games were played that day. Games are usually played between late May and mid October. If you can't find a game on the day you picked, try a different day until you find a game.

<br>Write a multiple conditional to find games played between July 7, 2012 and September 7, 2012.

## <br><br><br>7.3 Telling pandas to automatically turn dates into datetime objects when loading a file

We can use the `parse_dates` argument and pass it a list columns to become datetime objects.

In [ ]:
df = pd.read_csv("wnba-team-elo-ratings.csv", parse_dates=["date"])
print(df.dtypes)
df.head()

## <br><br><br>7.4 Using Date Column to Plot Trend Over Time

We can use the column with date data type to plot trend over time. Use the `.plot()` method of the DataFrame and set the date column as the x axis.

In [ ]:
df.plot(x="date", y="elo1_pre", figsize=(10, 5), title="Elo Ratings Over Time")

The plot above doesn't tell much since this shows progression across all teams. 

Let's take a look at elo rating of Chicago team over time.

In [ ]:
# Code to select chicago team
df.loc[df["team1"] == "CHI"].head()

In [ ]:
df.loc[df["team1"] == "CHI"].plot(x="date", y="elo1_pre", figsize=(10, 5), title="Elo Ratings of Chicago Sky Over Time")

### <br><br>Exercise 7.3

Try picking a different team and numeric variable to visualize.